## Atlas subsampling 

In [259]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [260]:
import os,sys
import scanpy as sc 
import pandas as pd
import numpy as np
import milopy
import scipy
import anndata

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

sns.set_context('talk')
from matplotlib.pyplot import rc_context

import milopy
import milopy.core as milo

## Load dataset

In [ ]:
data_dir = '/lustre/scratch117/cellgen/team205/ed6/PBMC_CZI_integration_filtered/'
adata = sc.read_h5ad(data_dir + 'PBMC_merged.normal.subsample500cells.clean_celltypes.h5ad')

## Make sample level anndata

In [265]:
sc.pp.neighbors(adata, use_rep='X_scVI', n_neighbors=100)

In [266]:
## Make nhoods
milo.make_nhoods(adata, prop=0.01)
milo.count_nhoods(adata, sample_col='donor_id')
milopy.utils.add_covariate_to_nhoods_var(adata, ['sex', 'dataset_id', 'ethnicity', 'assay'])
sample_adata = adata.uns['nhood_adata'].T.copy()
sample_adata.layers['counts'] = sample_adata.X.copy()

## Normalize counts x donor
sc.pp.normalize_total(sample_adata, target_sum=10000)
sc.pp.log1p(sample_adata)

## Dim reduction
sc.pp.pca(sample_adata)

## Compute similarity between donors
n_pcs = 30
X_pca = sample_adata.obsm['X_pca']
sample_adata.obsp['distance_full'] = scipy.spatial.distance.cdist(X_pca[:,0:n_pcs], X_pca[:,0:n_pcs])

/home/jovyan/my-conda-envs/emma_env/lib/python3.7/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [267]:
sample_adata.write_h5ad(data_dir + 'PBMC_merged.normal.subsample500cells.clean_celltypes.donor_adata.h5ad')

In [505]:
sample_adata = sc.read_h5ad(data_dir + 'PBMC_merged.normal.subsample500cells.clean_celltypes.donor_adata.h5ad')

### Compare controls 

In [506]:
## Define query donors
np.random.seed(41)
query_dataset = '10_1038_s41591_021_01329_2'
samples_design = np.random.choice((0, 1), sum(sample_adata.obs['dataset_id'] == query_dataset))
query_samples = sample_adata.obs_names[sample_adata.obs['dataset_id'] == query_dataset][samples_design == 1]


In [507]:
## Find set of closest samples to subset atlas
def find_ctrl_samples(
    sample_adata, 
    ctrl_samples_all,
    query_samples, 
    method = 'closest', ## one of closest, farthest, random
    n_ctrl: int = 15,
    random_seed = 42,
    n_pcs = 30
    ):
    '''
    Params:
    ------
    - sample_adata: sample-level anndata object
    - query_samples: list or collection of samples from query dataset
    - n_ctrl: number of control samples to pick
    '''
    
    # Get distances to query samples
    query_distances = scipy.spatial.distance.cdist(sample_adata[query_samples,:].obsm['X_pca'][:,0:n_pcs], sample_adata[ctrl_samples_all,:].obsm['X_pca'][:,0:n_pcs])
    
    # Get n_ctrl closest samples
    if method == 'closest':
        n_closest = 0
        k = 1
        ctrl_samples = np.array([])
        while n_closest < n_ctrl:
            closest_samples_k = np.argsort(query_distances, axis=1)[:,0:k]
            closest_samples = np.setdiff1d(np.unique(closest_samples_k), ctrl_samples)
            closest_dist = query_distances[:,closest_samples].min(axis=0).argsort()
            ctrl_samples = np.hstack([ctrl_samples, closest_samples[closest_dist][0:(n_ctrl - n_closest)]])
            n_closest = len(ctrl_samples)
            k +=1
        ctrl_samples = ctrl_samples_all[ctrl_samples.astype('int')]

    if method == 'farthest':
        n_closest = 0
        k = 1
        ctrl_samples = np.array([])
        while n_closest < n_ctrl:
            closest_samples_k = np.argsort(query_distances, axis=1)[:,-k:]
            closest_samples = np.setdiff1d(np.unique(closest_samples_k), ctrl_samples)
            closest_dist = query_distances[:,closest_samples].min(axis=0).argsort()
            ctrl_samples = np.hstack([ctrl_samples, closest_samples[closest_dist][0:(n_ctrl - n_closest)]])
            n_closest = len(ctrl_samples)
            k +=1
        ctrl_samples = ctrl_samples_all[ctrl_samples.astype('int')]

    if method == 'random':
        np.random.seed(random_seed)
        ctrl_samples = np.random.choice(ctrl_samples_all, n_ctrl, replace=False)

    return(ctrl_samples)
# ## Get ctrl samples closest to avg of query samples
# closest_samples = ctrl_samples[np.argsort(query_distances.mean(axis=0))[0:n_ctrl]]

In [508]:
## Matched control samples
matched_ctrl = sample_adata.obs_names[(sample_adata.obs['dataset_id'] == query_dataset) & (~sample_adata.obs_names.isin(query_samples))]
sample_adata.obs[f'matched_ctrl'] = sample_adata.obs_names.isin(closest_ctrl).astype('int')

ctrl_samples_all = sample_adata.obs_names[sample_adata.obs['dataset_id'] != query_dataset]
n_ctrl = len(matched_ctrl)

## Closest control samples
closest_ctrl = find_ctrl_samples(sample_adata, ctrl_samples_all, query_samples, method='closest', n_ctrl=n_ctrl)
sample_adata.obs[f'close_ctrl'] = sample_adata.obs_names.isin(closest_ctrl).astype('int')

## Farthest control samples
far_ctrl = find_ctrl_samples(sample_adata, ctrl_samples_all, query_samples, method='farthest', n_ctrl=n_ctrl)
sample_adata.obs[f'far_ctrl'] = sample_adata.obs_names.isin(far_ctrl).astype('int')

## Random control samples (x 3)
for i in [1,2,3]:
    random_ctrl = find_ctrl_samples(sample_adata, ctrl_samples_all, query_samples, n_ctrl=n_ctrl, method = 'random', random_seed = 42+i)
    sample_adata.obs[f'random{i}_ctrl'] = sample_adata.obs_names.isin(random_ctrl).astype('int')

View of AnnData object with n_obs × n_vars = 16 × 5916
    obs: 'ethnicity', 'sex', 'dataset_id', 'assay', 'donor_id', 'matched_ctrl', 'close_ctrl', 'far_ctrl', 'random1_ctrl', 'random2_ctrl', 'random3_ctrl'
    var: 'index_cell', 'kth_distance'
    uns: 'log1p', 'pca', 'sample_col'
    obsm: 'X_pca'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distance_full'

In [411]:
closest_dist = query_distances[:,closest_samples].min(axis=0).argsort()
closest_samples[closest_dist][0:n_ctrl - n_closest]

array([1046, 1093, 1007,   28, 1095])